### anexo 1 
Con estos campos, podemos extraer la información esencial de cada alerta. A continuación, se muestra cómo cargar el JSON de ZAP y extraer dichos campos en Python:

In [22]:
import json


# Cargar el reporte OWASP ZAP desde un archivo JSON
with open('ReporteZAP.json', 'r', encoding='utf-8') as f:
    zap_report = json.load(f)

# La estructura del JSON tiene un arreglo 'site' con sitios escaneados.
# Extraemos el primer (o único) sitio:
site = zap_report['site'][0]

# Obtener lista de alertas de seguridad
alerts = site['alerts']

# Inicializar lista para almacenar datos relevantes de cada alerta
vuln_data = []

for alert in alerts:
    # Extraer campos relevantes de la alerta
    cwe_id = alert.get('cweid')             # CWE ID (entero, 0 si no tiene)
    description = alert.get('desc', '')     # Descripción de la vulnerabilidad (con HTML)
    solution = alert.get('solution', '')    # Solución o recomendación sugerida (con HTML)
    riskdesc = alert.get('riskdesc', '')    # Riesgo + confianza, ej: "Medium (Low)"
    uri = '' 
    # Extraer una URI de ejemplo (la primera instancia de la vulnerabilidad)
    if 'instances' in alert and alert['instances']:
        uri = alert['instances'][0].get('uri', '')
    # Separar el nivel de riesgo del campo combinado (antes del paréntesis)
    risk = riskdesc.split('(')[0].strip() if riskdesc else ''

    # Almacenar los datos en un diccionario
    vuln_data.append({
        'cweid': cwe_id,
        'desc': description,
        'solution': solution,
        'uri': uri,
        'risk': risk
    })

# Revisar cuántas vulnerabilidades se extrajeron
print(f"Vulnerabilidades extraídas: {len(vuln_data)}")
# Opcional: mostrar un ejemplo de los datos extraídos (antes de limpieza)
print(vuln_data[0])

jdown = json.dumps(vuln_data, indent=4, ensure_ascii=False)
# Guardar los datos extraídos en un archivo JSON        
json_file = 'vul_Norm.json'
with open(json_file, 'w', encoding='utf-8') as f:
    f.write(jdown)
    


Vulnerabilidades extraídas: 17
{'cweid': '352', 'desc': "<p>No Anti-CSRF tokens were found in a HTML submission form.</p><p>A cross-site request forgery is an attack that involves forcing a victim to send an HTTP request to a target destination without their knowledge or intent in order to perform an action as the victim. The underlying cause is application functionality using predictable URL/form actions in a repeatable way. The nature of the attack is that CSRF exploits the trust that a web site has for a user. By contrast, cross-site scripting (XSS) exploits the trust that a user has for a web site. Like XSS, CSRF attacks are not necessarily cross-site, but they can be. Cross-site request forgery is also known as CSRF, XSRF, one-click attack, session riding, confused deputy, and sea surf.</p><p></p><p>CSRF attacks are effective in a number of situations, including:</p><p>    * The victim has an active session on the target site.</p><p>    * The victim is authenticated via HTTP auth 

In [ ]:
import json
from pathlib import Path

class ProcesadorReporteZAP:
    def __init__(self, archivo_entrada='ReporteZAP.json', archivo_salida='vul_Norm.json'):
        self.archivo_entrada = Path(archivo_entrada)
        self.archivo_salida = Path(archivo_salida)
        
    def cargar_reporte(self):
        try:
            with open(self.archivo_entrada, 'r', encoding='utf-8') as f:
                return json.load(f)
        except Exception as e:
            raise ValueError(f"Error cargando reporte: {str(e)}")
            
    def procesar_alerta(self, alert):
        return {
            'cweid': alert.get('cweid'),
            'desc': alert.get('desc', ''),
            'solution': alert.get('solution', ''),
            'uri': self._obtener_uri(alert),
            'risk': self._extraer_nivel_riesgo(alert.get('riskdesc', ''))
        }
        
    def _obtener_uri(self, alert):
        return alert['instances'][0].get('uri', '') if 'instances' in alert and alert['instances'] else ''
        
    def _extraer_nivel_riesgo(self, riskdesc):
        return riskdesc.split('(')[0].strip() if riskdesc else ''
        
    def procesar(self):
        try:
            zap_report = self.cargar_reporte()
            site = zap_report['site'][0]
            vuln_data = [self.procesar_alerta(alert) for alert in site['alerts']]
            
            with open(self.archivo_salida, 'w', encoding='utf-8') as f:
                json.dump(vuln_data, f, indent=4, ensure_ascii=False)
                
            return len(vuln_data)
        except Exception as e:
            raise RuntimeError(f"Error procesando reporte: {str(e)}")

# Uso básico
procesador = ProcesadorReporteZAP()
vulnerabilidades = procesador.procesar()

# Uso avanzado con archivos personalizados
procesador_custom = ProcesadorReporteZAP(
    archivo_entrada='mi_reporte.json',
    archivo_salida='resultado_personalizado.json'
)

Explicación: En este fragmento, se carga el archivo JSON y se itera sobre cada alerta dentro de site['alerts']. Para cada alerta, se obtienen los campos mencionados. Observamos que riskdesc incluye tanto el riesgo como la confianza (por ejemplo "Medium (Low)" indica riesgo Medio con confianza Baja​
zaproxy.org
). Dividimos esa cadena para quedarnos solo con el nivel de riesgo en risk. También obtenemos la primera uri de la lista de instancias donde se encontró la vulnerabilidad. Finalmente, almacenamos los datos en un diccionario por alerta y recopilamos todos en la lista vuln_data. el cual posteriormente se guarda en un archivo json llamado vul_Norm.json

### anexo 2
Usaremos bibliotecas Python para estas tareas: re para expresiones regulares (eliminar HTML y caracteres especiales) y nltk para la lematización (WordNet lemmatizer en inglés). Antes de ejecutar, aseguramos tener los recursos de NLTK necesarios (por ejemplo, WordNet). A continuación se presenta una función de limpieza y su aplicación a los datos:

In [ ]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
# Descargar recursos necesarios de NLTK (descomentar si es la primera vez)
# nltk.download('punkt')
# nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def limpiar_texto(texto):
    # 1. Eliminar etiquetas HTML usando una expresión regular
    texto_limpio = re.sub(r'<[^>]+>', ' ', texto)
    # 2. Convertir a minúsculas
    texto_limpio = texto_limpio.lower()
    # 3. Eliminar caracteres especiales (cualquier cosa no alfanumérica o espacio)
    texto_limpio = re.sub(r'[^a-z0-9áéíóúüñç\s]', ' ', texto_limpio)
    # 4. Eliminar espacios extra
    texto_limpio = re.sub(r'\s+', ' ', texto_limpio).strip()
    # 5. Lematizar cada palabra del texto
    palabras = nltk.word_tokenize(texto_limpio)
    palabras_lematizadas = [lemmatizer.lemmatize(p) for p in palabras]
    texto_limpio = ' '.join(palabras_lematizadas)
    return texto_limpio

# Aplicar limpieza a la descripción y la solución de cada vulnerabilidad
for vuln in vuln_data:
    vuln['desc'] = limpiar_texto(vuln['desc'])
    vuln['solution'] = limpiar_texto(vuln.get('solution', ''))

# Ejemplo: mostrar descripción y solución limpias de la primera vulnerabilidad
print("Descripción limpia:", vuln_data[0]['desc'])
print("Recomendación limpia:", vuln_data[0]['solution'])

Explicación: Definimos la función limpiar_texto que aplica todas las transformaciones mencionadas. Primero, reemplazamos cualquier secuencia que luzca como etiqueta HTML por un espacio usando re.sub (regex r'<[^>]+>' captura < seguido de cualquier caracter no > hasta encontrar >). Luego convertimos a minúsculas con .lower(). Usamos otra expresión regular r'[^a-z0-9áéíóúüñç\s]' para eliminar caracteres no alfanuméricos (permitiendo letras, dígitos, espacios y algunos caracteres con acentos comunes en español). Esto quita signos de puntuación como <, >, /, etc., dejando solo texto limpio. Después contraemos múltiples espacios seguidos en uno solo y hacemos strip() para eliminar espacios iniciales/finales. Finalmente, usamos nltk.word_tokenize para separar el texto en palabras y aplicamos el lematizador de WordNet a cada palabra. La lematización reduce cada palabra a su forma base o lema (por ejemplo, plural a singular, verbos a infinitivo en inglés, etc.), lo cual unifica variantes de la misma palabra​
. El texto final se reconstruye uniendo las palabras lematizadas con espacios.

### anexo 3 
A continuación, construimos la nueva estructura de datos siguiendo estas reglas:

In [ ]:
import hashlib

diverse_data = []  # lista para almacenar las entradas en formato DiverseVul

# Obtener identificador del "proyecto" (sitio) desde el JSON de ZAP
project_name = site.get('@host') or site.get('@name') or 'Proyecto_ZAP'

for alert, vuln in zip(alerts, vuln_data):
    # Tomar el pluginid de ZAP como base de un ID (es un número string)
    plugin_id = alert.get('pluginid', '')
    if not plugin_id:
        plugin_id = 'unknown'
    # Usar pluginid como commitID (en string)
    commit_id = str(plugin_id)
    # Generar un hash único combinando sitio + pluginid + uri
    unique_str = f"{project_name}_{plugin_id}_{vuln['uri']}"
    hash_id = hashlib.md5(unique_str.encode('utf-8')).hexdigest()[:8]  # hash de 8 caracteres
    
    # Calcular tamaño como número de palabras de la descripción
    size = len(vuln['desc'].split())
    
    # Armar el diccionario con campos formato DiverseVul
    diverse_data.append({
        'func': vuln['desc'],             # texto de la "función" (descripción de vulnerabilidad)
        'target': 1,                      # 1 = vulnerable (todas las alertas lo son)
        'cwe': vuln['cweid'],             # ID CWE (entero)
        'project': project_name,          # nombre del proyecto/sitio
        'commitID': commit_id,            # identificador de commit (simulado con pluginid)
        'hash': hash_id,                  # hash único de la entrada
        'size': size,                     # tamaño de la "función" en número de palabras
        'message': vuln['solution']       # mensaje (solución recomendada)
    })

# Mostrar un ejemplo del nuevo formato
from pprint import pprint
print("Ejemplo de entrada en formato DiverseVul adaptado:")
(diverse_data[0])


Explicación: Aquí creamos la lista diverse_data donde cada elemento es un diccionario siguiendo el esquema de DiverseVul. Usamos site['@host'] (o @name) para identificar el proyecto (por ejemplo, "example.com"). Para commitID, asignamos el pluginid de la alerta (que identifica el tipo de vulnerabilidad en ZAP, por ejemplo 40012 para XSS reflejado). Generamos un hash MD5 a partir del nombre de proyecto, pluginid y URI (concatenados) para obtener un string único (tomamos los primeros 8 caracteres hexadecimales para legibilidad). El campo size es el número de palabras en la descripción limpia. Finalmente, message toma el texto de la solución recomendada.

### anexo 4
Configuración del tokenizador T5: Usaremos la implementación de HuggingFace Transformers. A continuación se muestra cómo inicializar el tokenizador de T5-base y cómo tokenizar un ejemplo de nuestros datos:

In [ ]:

from transformers import AutoTokenizer

# Inicializar el tokenizador para T5-base
tokenizer = AutoTokenizer.from_pretrained('t5-base')

# Tomar un ejemplo de entrada (func) y su etiqueta (cwe) del diverse_data
ejemplo = diverse_data[0]
input_text = ejemplo['func']        # texto de entrada (descripción de la vulnerabilidad limpia)
target_text = str(ejemplo['cwe'])   # texto de la etiqueta (CWE id en string)

# Tokenizar la entrada con límite de 512 tokens
enc_input = tokenizer(
    input_text,
    max_length=512,
    truncation=True,
    padding='max_length',   # rellenar hasta 512 para demostración (opcional, se puede usar padding dinámico en batch)
    return_tensors='pt'     # retornar tensores de PyTorch (opcional, para ilustrar)
)

# Tokenizar la salida/etiqueta con límite de 2 tokens
enc_target = tokenizer(
    target_text,
    max_length=2,
    truncation=True,
    padding='max_length',
    return_tensors='pt'
)

# Inspeccionar el resultado de tokenización
print("Tokens de entrada (ids) [parcial]:", enc_input['input_ids'][0, :10].tolist(), "...")
print("Longitud total tokens entrada:", (enc_input['input_ids'] != tokenizer.pad_token_id).sum().item())
print("Tokens de etiqueta (ids):", enc_target['input_ids'][0].tolist())
print("Longitud tokens etiqueta:", (enc_target['input_ids'] != tokenizer.pad_token_id).sum().item())


Explicación: Usamos AutoTokenizer.from_pretrained('t5-base') para cargar el tokenizador preentrenado de T5-base (esto descargará el vocabulario SentencePiece y la configuración del tokenizador). Luego, definimos input_text como el campo func de un ejemplo (que es la descripción de la vulnerabilidad) y target_text como el cwe convertido a string. Tokenizamos la entrada con max_length=512 y truncation=True para asegurarnos de no exceder 512 tokens – si la descripción fuera muy larga, se recortará. Usamos padding='max_length' aquí para rellenar la secuencia a 512 tokens (esto agregará tokens <pad> al final si el texto es más corto). En un caso real, podríamos preferir padding=True (que rellena solo hasta la longitud del texto más largo en el batch) para no añadir padding innecesario; pero para demostración fijamos el tamaño. También pedimos return_tensors='pt' para obtener tensores (aunque podríamos trabajar con las listas de IDs directamente; esto es útil si vamos a usar PyTorch). Repetimos el proceso para el target_text con max_length=2.

### anexo 5

A continuación, convertimos todas las muestras de diverse_data a sus representaciones tokenizadas y preparamos estructuras para entrenamiento:

In [ ]:
# Preparar listas para almacenar los tensores/ids de entradas y etiquetas
input_ids_list = []
attention_mask_list = []
label_ids_list = []

for item in diverse_data:
    # Texto de entrada y texto de salida
    text = item['func']
    label = str(item['cwe'])
    # Tokenizar entrada
    enc_in = tokenizer(text, max_length=512, truncation=True, padding='max_length')
    # Tokenizar etiqueta
    enc_out = tokenizer(label, max_length=2, truncation=True, padding='max_length')
    # Extraer ids y mask
    input_ids_list.append(enc_in['input_ids'])
    attention_mask_list.append(enc_in['attention_mask'])
    # Para las etiquetas, reemplazar pad token id por -100 (ignorado en loss) si es necesario
    label_ids = enc_out['input_ids']
    # Reemplazar <pad> (id=0 en T5) con -100 en la etiqueta
    label_ids = [(-100 if token_id == tokenizer.pad_token_id else token_id) for token_id in label_ids]
    label_ids_list.append(label_ids)

# Convertir a tensores de PyTorch (opcional) para uso directo en modelos/entrenador
import torch
input_ids_tensor = torch.tensor(input_ids_list)
attention_mask_tensor = torch.tensor(attention_mask_list)
labels_tensor = torch.tensor(label_ids_list)

print("Shape de input_ids:", input_ids_tensor.shape)
print("Shape de attention_mask:", attention_mask_tensor.shape)
print("Shape de labels:", labels_tensor.shape)


Explicación: Recorremos cada elemento de diverse_data y aplicamos el tokenizador. En este caso, usamos el tokenizador en modo no-tensor (por defecto devuelve listas de IDs) para luego convertir todo junto a tensores al final. Para cada entrada, truncamos/pad a 512 tokens. Para cada etiqueta, truncamos/pad a 2 tokens. Al obtener los input_ids de la etiqueta, hacemos una post-procesamiento: reemplazamos cualquier token ID que corresponda a <pad> por -100. Esto sigue la convención de training en HuggingFace donde la pérdida ignora posiciones con etiqueta -100. Como en T5 el token de padding tiene id 0, estaremos sustituyendo esos 0 en la secuencia de la etiqueta. Luego, almacenamos las listas de IDs.